In [1]:
import os
os.chdir("../..")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from utils.dataset_loader import load_datasets
from utils.model_utils import initialize_model
from utils.train_utils import train_model
from utils.metrics import evaluate_model
from utils.visualization import plot_training, plot_confusion_matrix
import optuna
import os

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

# Ensure output directories exist
os.makedirs("outputs/models", exist_ok=True)
os.makedirs("outputs/plots", exist_ok=True)

# Load Dataset
print("[INFO] Loading datasets...")
train_loader, val_loader, test_loader = load_datasets(
    "wildfire_dataset_scaled", batch_size=32, augmentation="augmented"
)
print("[INFO] Datasets loaded successfully!")

# Objective function for Optuna
def objective(trial):
    print(f"[DEBUG] Starting trial {trial.number}...")

    # Hyperparameter suggestions
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "AdamW"])
    print(f"[DEBUG] Suggested hyperparameters: lr={lr}, weight_decay={weight_decay}, optimizer={optimizer_name}")

    # Initialize model
    print("[DEBUG] Initializing VGG16 model...")
    model = initialize_model("vgg16", num_classes=2, pretrained=True, freeze_all=False, unfreeze_last_n=2)
    model.to(device)

    # Optimizer setup
    print("[DEBUG] Setting up optimizer...")
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=3, factor=0.5, verbose=True)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Train the model
    print("[INFO] Starting model training...")
    save_path = "outputs/models/vgg16_trial_best.pth"
    history = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        save_path=save_path,
        early_stop_patience=5,
        monitor_metric="val_recall",
    )
    print("[INFO] Training completed!")

    # Return validation recall
    val_recall = history["val_recall"][-1]
    print(f"[INFO] Trial {trial.number} - Final Validation Recall: {val_recall:.4f}")
    return val_recall

# Run Optuna
print("[INFO] Running hyperparameter optimization...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
print(f"[INFO] Best parameters found: {study.best_params}")

# Train Final Model
print("[INFO] Training final model with best hyperparameters...")
final_model = initialize_model("vgg16", num_classes=2, pretrained=True, freeze_all=False, unfreeze_last_n=2)
final_model.to(device)

# Optimizer
optimizer = optim.Adam(final_model.parameters(), lr=study.best_params["lr"], weight_decay=study.best_params["weight_decay"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=3, factor=0.5)

# Final training
final_save_path = "outputs/models/vgg16_final.pth"
history = train_model(
    model=final_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=nn.CrossEntropyLoss(),
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    save_path=final_save_path,
    early_stop_patience=5,
    monitor_metric="val_recall",
)

# Evaluation and Visualization
print("[INFO] Evaluating final model...")
metrics = evaluate_model(final_model, test_loader, ["No Fire", "Fire"], device)
plot_training(history, "outputs/plots/vgg16_training_curve.png")
plot_confusion_matrix(metrics["confusion_matrix"], ["No Fire", "Fire"], "outputs/plots/vgg16_confusion_matrix.png")
print("[INFO] Final results saved successfully!")


c:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
[I 2024-12-18 18:24:17,085] A new study created in memory with name: no-name-52f71643-e652-4a1b-9e59-078e487076a2


[INFO] Using device: cuda
[INFO] Loading datasets...
[INFO] Datasets loaded successfully!
[INFO] Running hyperparameter optimization...
[DEBUG] Starting trial 0...
[DEBUG] Suggested hyperparameters: lr=4.1098689333400045e-05, weight_decay=1.1050285411686587e-05, optimizer=Adam
[DEBUG] Initializing VGG16 model...


C:\Users\Yash\AppData\Local\Temp\ipykernel_13848\976140276.py:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
C:\Users\Yash\AppData\Local\Temp\ipykernel_13848\976140276.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-3)
c:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\to

[DEBUG] Setting up optimizer...
[INFO] Starting model training...

Starting training...



Epoch [1] - Training:  80%|███████▉  | 47/59 [01:32<00:24,  2.07s/it]